In [1]:
import pandas as pd
import sys

sys.path.insert(0, "../../src")
from utils import *
from run_mp import *
from gee_data_wrangling import *
from gee_data_download import *

*** Earth Engine *** Share your feedback by taking our Annual Developer Satisfaction Survey: https://google.qualtrics.com/jfe/form/SV_doiqkQG3NJ1t8IS?source=API


In [2]:
all_sites = pd.read_csv(here("python/00_process_nfi_data/nfi_final_sites.csv"))
sites_subset = all_sites.copy()

if TAKE_ONLY_10k_SITES:
    sites_subset = all_sites.copy().iloc[:10000, :]

sites_subset
# grouped = sites_subset.groupby("id", as_index=False)
# df_list = [group for name, group in grouped]

# df_list[0]

NameError: name 'TAKE_ONLY_10k_SITES' is not defined

In [3]:
TAKE_ONLY_10k_SITES = True

# Loop to get all data and merge it
my_datasets = ["era5-daily", "modis-vi", "sentinel", "landsat", "modis-lst-terra"]
df_all = pd.DataFrame(columns=["SiteID", "first_year"])

for my_dataset in my_datasets:
    print(f"\n> Working on {my_dataset}")

    # Load all sites
    all_sites = pd.read_csv(here("python/00_process_nfi_data/nfi_final_sites.csv"))
    sites_subset = all_sites.copy()

    if TAKE_ONLY_10k_SITES:
        sites_subset = all_sites.copy().iloc[:10000, :]

    grouped = sites_subset.groupby("SiteID", as_index=False)
    df_list = [group for name, group in grouped]

    out = run_mp(
        load_and_wrangle_PARALLEL,
        df_list,
        combine_func=pd.concat,
        progress_bar=True,
        num_cores=10,
        subdir=my_dataset,
        verbose=False,
        return_which="wrangled",
    )

    df_all = pd.merge(df_all, out, how="outer", on=["SiteID", "first_year"])


> Working on era5-daily


100%|██████████| 10000/10000 [01:08<00:00, 146.87it/s]



> Working on modis-vi


100%|██████████| 10000/10000 [00:12<00:00, 818.63it/s]



> Working on sentinel


100%|██████████| 10000/10000 [00:14<00:00, 667.36it/s]



> Working on landsat


100%|██████████| 10000/10000 [00:14<00:00, 687.69it/s]



> Working on modis-lst-terra


100%|██████████| 10000/10000 [00:04<00:00, 2204.72it/s]


In [4]:
df_all

,SiteID,first_year,max_gdd_before_spring_frost,min_doy_of_fall_frost,hw_counts,hw_dur_max,hw_dur_mean,hw_day_sum,hw_days_between,hw_mean_temp,...,mean_of_landsat_EVI_in_fall,mean_of_landsat_EVI_in_winter,std_of_landsat_NDVI_in_spring,std_of_landsat_NDVI_in_summer,std_of_landsat_NDVI_in_fall,std_of_landsat_NDVI_in_winter,std_of_landsat_EVI_in_spring,std_of_landsat_EVI_in_summer,std_of_landsat_EVI_in_fall,std_of_landsat_EVI_in_winter
0,1,2012,95.0,334.0,0.0,NaN,NaN,NaN,NaN,NaN,...,0.404796,0.243736,0.042461,0.019333,0.028528,0.044926,0.066791,0.038358,0.048300,0.061198
1,2,2012,85.0,284.0,0.0,NaN,NaN,NaN,NaN,NaN,...,0.470416,0.261726,0.115492,0.058408,0.089923,0.026370,0.193583,0.121745,0.157663,0.040017
2,3,2012,88.0,302.0,0.0,NaN,NaN,NaN,NaN,NaN,...,0.456291,0.318117,0.094763,0.078777,0.052318,0.098002,0.155626,0.128103,0.084604,0.139906
3,4,2012,91.0,308.0,0.0,NaN,NaN,NaN,NaN,NaN,...,0.444941,0.437292,0.064986,0.048748,0.034503,0.061557,0.115424,0.089904,0.054856,0.093915
4,5,2012,58.0,314.0,0.0,NaN,NaN,NaN,NaN,NaN,...,0.383762,0.370150,0.035876,0.019348,0.042045,0.050371,0.059088,0.031746,0.056875,0.062398
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9994,9995,2010,60.0,301.0,0.0,NaN,NaN,NaN,NaN,NaN,...,0.380072,0.331740,0.034374,0.031699,0.046533,0.091550,0.057763,0.056062,0.078950,0.136502
9995,9996,2010,64.0,303.0,0.0,NaN,NaN,NaN,NaN,NaN,...,0.515430,0.286057,0.003224,0.021186,0.103936,NaN,0.006158,0.042790,0.184315,NaN
9996,9997,2010,55.0,294.0,0.0,NaN,NaN,NaN,NaN,NaN,...,0.351470,NaN,0.084695,0.076561,0.069641,NaN,0.139310,0.127054,0.094897,NaN
9997,9998,2010,80.0,294.0,0.0,NaN,NaN,NaN,NaN,NaN,...,0.348353,0.288838,0.062849,0.049175,0.026973,0.030694,0.108156,0.076197,0.044814,0.049092


In [9]:
df_all.to_feather("final_gee_predictor_dataset.feather")
df_all.to_csv("final_gee_predictor_dataset.csv", index=False)

In [6]:
pd.read_feather("final_gee_predictor_dataset.feather")

,SiteID,first_year,max_gdd_before_spring_frost,min_doy_of_fall_frost,hw_counts,hw_dur_max,hw_dur_mean,hw_day_sum,hw_days_between,hw_mean_temp,...,mean_of_landsat_EVI_in_fall,mean_of_landsat_EVI_in_winter,std_of_landsat_NDVI_in_spring,std_of_landsat_NDVI_in_summer,std_of_landsat_NDVI_in_fall,std_of_landsat_NDVI_in_winter,std_of_landsat_EVI_in_spring,std_of_landsat_EVI_in_summer,std_of_landsat_EVI_in_fall,std_of_landsat_EVI_in_winter
0,1,2011,71.0,NaN,0,NaN,NaN,NaN,NaN,NaN,...,0.409092,0.243736,0.042461,0.019333,0.020443,0.044926,0.066791,0.038358,0.034453,0.061198
1,2,2012,85.0,284.0,0,NaN,NaN,NaN,NaN,NaN,...,0.470416,0.261726,0.115492,0.058408,0.089923,0.026370,0.193583,0.121745,0.157663,0.040017
2,3,2012,88.0,302.0,0,NaN,NaN,NaN,NaN,NaN,...,0.456291,0.318117,0.094763,0.078777,0.052318,0.098002,0.155626,0.128103,0.084604,0.139906
3,4,2012,91.0,308.0,0,NaN,NaN,NaN,NaN,NaN,...,0.444941,0.437292,0.064986,0.048748,0.034503,0.061557,0.115424,0.089904,0.054856,0.093915
4,5,2012,58.0,314.0,0,NaN,NaN,NaN,NaN,NaN,...,0.383762,0.370150,0.035876,0.019348,0.042045,0.050371,0.059088,0.031746,0.056875,0.062398
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38297,38298,2016,51.0,300.0,0,NaN,NaN,NaN,NaN,NaN,...,0.320209,0.257313,0.066885,0.040937,0.063039,0.042583,0.106770,0.076283,0.092620,0.063518
38298,38299,2016,68.0,304.0,0,NaN,NaN,NaN,NaN,NaN,...,0.371641,0.175064,0.109530,0.043192,0.066637,0.013540,0.180242,0.087778,0.107406,0.021669
38299,38300,2016,51.0,NaN,0,NaN,NaN,NaN,NaN,NaN,...,0.423844,0.418582,0.023636,0.032380,0.071105,0.031718,0.039081,0.051603,0.110863,0.081013
38300,38301,2016,67.0,284.0,0,NaN,NaN,NaN,NaN,NaN,...,0.397496,0.303379,0.069576,0.034570,0.020568,0.087935,0.110179,0.061193,0.039139,0.088243


In [2]:
pd.read_csv("final_gee_predictor_dataset.csv")

,SiteID,first_year,max_gdd_before_spring_frost,min_doy_of_fall_frost,hw_counts,hw_dur_max,hw_dur_mean,hw_day_sum,hw_days_between,hw_mean_temp,...,mean_of_landsat_EVI_in_fall,mean_of_landsat_EVI_in_winter,std_of_landsat_NDVI_in_spring,std_of_landsat_NDVI_in_summer,std_of_landsat_NDVI_in_fall,std_of_landsat_NDVI_in_winter,std_of_landsat_EVI_in_spring,std_of_landsat_EVI_in_summer,std_of_landsat_EVI_in_fall,std_of_landsat_EVI_in_winter
0,1,2011,71.0,NaN,0,NaN,NaN,NaN,NaN,NaN,...,0.409092,0.243736,0.042461,0.019333,0.020443,0.044926,0.066791,0.038358,0.034453,0.061198
1,2,2012,85.0,284.0,0,NaN,NaN,NaN,NaN,NaN,...,0.470416,0.261726,0.115492,0.058408,0.089923,0.026370,0.193583,0.121745,0.157663,0.040017
2,3,2012,88.0,302.0,0,NaN,NaN,NaN,NaN,NaN,...,0.456291,0.318117,0.094763,0.078777,0.052318,0.098002,0.155626,0.128103,0.084604,0.139906
3,4,2012,91.0,308.0,0,NaN,NaN,NaN,NaN,NaN,...,0.444941,0.437292,0.064986,0.048748,0.034503,0.061557,0.115424,0.089904,0.054856,0.093915
4,5,2012,58.0,314.0,0,NaN,NaN,NaN,NaN,NaN,...,0.383762,0.370150,0.035876,0.019348,0.042045,0.050371,0.059088,0.031746,0.056875,0.062398
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38297,38298,2016,51.0,300.0,0,NaN,NaN,NaN,NaN,NaN,...,0.320209,0.257313,0.066885,0.040937,0.063039,0.042583,0.106770,0.076283,0.092620,0.063518
38298,38299,2016,68.0,304.0,0,NaN,NaN,NaN,NaN,NaN,...,0.371641,0.175064,0.109530,0.043192,0.066637,0.013540,0.180242,0.087778,0.107406,0.021669
38299,38300,2016,51.0,NaN,0,NaN,NaN,NaN,NaN,NaN,...,0.423844,0.418582,0.023636,0.032380,0.071105,0.031718,0.039081,0.051603,0.110863,0.081013
38300,38301,2016,67.0,284.0,0,NaN,NaN,NaN,NaN,NaN,...,0.397496,0.303379,0.069576,0.034570,0.020568,0.087935,0.110179,0.061193,0.039139,0.088243


In [3]:
df_all = pd.read_csv("final_gee_predictor_dataset.csv")

In [6]:
df_sites = pd.read_csv("../00_process_nfi_data/nfi_final_sites.csv", index_col=0)

In [7]:
pd.read_csv(here("python/00_process_nfi_data/nfi_final_sites.csv"))

,first_visit,id,x,y
0,2011,1,-2.842824,48.337505
1,2012,2,3.349757,46.198025
2,2012,3,3.361577,46.827747
3,2012,4,0.402182,48.201563
4,2012,5,6.461081,43.281648
...,...,...,...,...
38297,2016,38298,2.156438,42.686748
38298,2016,38299,7.424664,47.899971
38299,2016,38300,-3.221877,48.851908
38300,2016,38301,3.757202,44.924973


In [4]:
# Load data from csv
data = get_nfi_coordinates_site_year(with_idp=False)

# Clean first and last date for download routine
data_clean = adjust_first_last_date(
    df=data,
    plus_years=1,
    minus_years=1,
    first_date="01-01",
    last_date="12-31",
)

KeyError: 'first_visit'

In [2]:
get_nfi_coordinates_site_year(with_idp=False)

,SiteID,idp,first_year,x,y
0,0,632691,2011,-2.842824,48.337505
1,1,702597,2012,3.349757,46.198025
2,2,706240,2012,3.361577,46.827747
3,3,708321,2012,0.402182,48.201563
4,4,708369,2012,6.461081,43.281648
...,...,...,...,...,...
38691,38691,1131409,2016,2.156438,42.686748
38692,38692,1131410,2016,7.424664,47.899971
38693,38693,1131419,2016,-3.221877,48.851908
38694,38694,1131424,2016,3.757202,44.924973
